In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# read data to panads 
data = pd.read_csv('./podcast18.csv') 

# read and convert data schema mapping into pandas
data_schema = pd.read_csv('./data_schema.csv')

#dict of mapping drived from data schema file
mapping = {}
# getting unique mapping of varible and lable
data_schema_unique = data_schema.groupby(['Variable', 'cleaned_label']).size().reset_index(name='Freq')
# use mapping dict to create a key value pair with varible as key and lable as value
for index, row in data_schema_unique.iterrows():
    if row['cleaned_label']:
        mapping[row['Variable']] = row['cleaned_label']

# use the mapping dict to rename the varibles of data file with our newly created shchema
data.rename(columns=mapping,inplace=True)

In [4]:
# replace all whitespace with NaN values
data.replace(' ', np.nan, inplace=True)

In [5]:
# drop all empty columns
data.dropna(axis = 1, how= 'all', inplace = True)

In [6]:
#drop columns with 20% NaN or more
data = data.loc[:, data.isnull().mean() <= .50]

In [7]:
#drop unnecessary columns based on domain knowledge
data = data.drop(['study','arfkid .KIDS IN HH','<none>','0','PanelistCulture','qquotas .QUOTA','arfrr .REGION ROLLUP','qp10ba .PROPORTION- FRENCH LANGUAGE','qp10bb .PROPORTION- ENGLISH LANGUAGE','qp10bc .PROPORTION- ANOTHER LANGUAGE','qp10bd .PROPORTION- TOTAL'],axis=1)

In [8]:
#replace _ with space in column names
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [9]:
data = data.drop(['r_entirely_responsibl_ly_in_the_responsibil_household_are_mostly_ponsibility_for_this'], axis=1)

In [10]:
#Drop all columns that contain same data (Standard deviation is 0)
data = data.drop(data.std()[(data.std() == 0)].index, axis=1)

In [11]:
 mapping = { data.columns[2]:'gender', data.columns[0]:'respondent_id', data.columns[1]:'region', data.columns[3]:'Age',data.columns[4]:'household_size',data.columns[5]:'highest_education',data.columns[6]:'employment_status',data.columns[7]:'salary',data.columns[8]:'occupation'}
 data = data.rename(columns=mapping)

In [12]:
# cleaning age column
data.Age = data.Age.apply(lambda x: '65' if '65+' in x else x)
data.Age = data.Age.apply(lambda x: '18' if 'Under 18' in x else x)
data.Age = data.Age.apply(lambda x: '50' if '45 to 54' in x else x)
data.Age = data.Age.apply(lambda x: '21' if '18 to 24' in x else x)
data.Age = data.Age.apply(lambda x: '40' if '35 to 44' in x else x)
data.Age = data.Age.apply(lambda x: '60' if '55 to 64' in x else x)
data.Age = data.Age.apply(lambda x: '30' if '25 to 34' in x else x)


In [13]:
# clean count of podcast listened to per week
data.count_of_podcasts_over_p4 = data.count_of_podcasts_over_p4.apply(lambda x: '0' if 'None' in x else x)
nonzero_mean_count_of_podcasts_over_p4 = data['count_of_podcasts_over_p4'][ data.household_size != 0 ].mean()
data.loc[data.count_of_podcasts_over_p4 == 0, "count_of_podcasts_over_p4" ] = nonzero_mean_count_of_podcasts_over_p4

In [14]:
data.household_size = data.household_size.apply(lambda x: '0' if 'Dk/Na' in x else x)
data['household_size'].astype(int)
nonzero_mean_household_size = data['household_size'][ data.household_size != 0 ].mean()
data.loc[data.household_size == 0, "household_size" ] = nonzero_mean_household_size

In [15]:
# Clean salary column
data['salary'] = data['salary'].str.extract('.*\((.*)\).*')
data['salary'] = data['salary'].fillna(value=0).astype(int)
nonzero_mean = data['salary'][ data.salary != 0 ].mean()
data.loc[data.salary == 0, "salary" ] = nonzero_mean

In [16]:
# change data type from o to int
data['salary'] = data['salary'].astype(int)
data['Age'] = data['Age'].astype(int)
data['household_size'] = data['household_size'].astype(int)
data['count_of_podcasts_over_p4'] = data['count_of_podcasts_over_p4'].astype(int)

In [17]:
# replace Nan Values within the numeric df
numeric_data = data.select_dtypes(include=['int64','int32']).fillna(data.mean()).astype(int)

In [18]:
numeric_data = numeric_data.rename(columns={'count_of_podcasts_over_p4':'count_listen_podcasts_perweek','qp10aa_.proportion-_canada':'canadian_podcast_ratio_listen','qp10ab_.proportion-_the_u.s.':'us_podcast_ratio_listen','qp10ac_.proportion-_britain':'britain_podcast_ratio_listen','qp10ad_.proportion-_france':'french_podcast_ratio_listen','qp10ae_.proportion-_elsewhere':'elsewhere_podcast_ratio_listen'})

In [19]:
data = data.rename(columns={'qp4gnaa_.past_month0_genre':'genre'})

In [20]:
categorical_data = data.select_dtypes(include=['object'])

### Create two new data frames

1. Categorical data (categoical_data)
2. KYC data (kyc_data)
3. numeric data (numeric_data)

In [21]:
# creating categorical data frame 
categorical_data = categorical_data[['region','gender','highest_education','employment_status','occupation','genre']]
cat_resp_id = numeric_data['respondent_id']
categorical_data = pd.concat([categorical_data, cat_resp_id], axis=1).reindex(categorical_data.index)

In [22]:
#moves responded id column to front
cols = categorical_data.columns.tolist()
n = int(cols.index('respondent_id'))
cols = [cols[n]] + cols[:n] + cols[n+1:]
categorical_data = categorical_data[cols]

In [23]:
#creates a random permuation of the categorical values
permutation = np.random.permutation(categorical_data['genre'])

#erase the empty values
empty_is = np.where(permutation == "")
permutation = np.delete(permutation, empty_is)

#replace all empty values of the dataframe[field]
end = len(permutation)
categorical_data['genre'] = categorical_data['genre'].apply(lambda x: permutation[np.random.randint(end)] if pd.isnull(x) else x)

In [24]:
#creates a random permuation of the categorical values
permutation = np.random.permutation(categorical_data['genre'])

#erase the empty values
empty_is = np.where(permutation == "")
permutation = np.delete(permutation, empty_is)

#replace all empty values of the dataframe[field]
end = len(permutation)
categorical_data['genre'] = categorical_data['genre'].apply(lambda x: permutation[np.random.randint(end)] if pd.isnull(x) else x)

In [25]:
#create a kyc dataframe
num_kyc_data = numeric_data[['Age','salary','household_size']]
kyc_data = pd.concat([categorical_data, num_kyc_data], axis=1).reindex(categorical_data.index)

In [26]:
kyc_data.dtypes

respondent_id         int32
region               object
gender               object
highest_education    object
employment_status    object
occupation           object
genre                object
Age                   int32
salary                int32
household_size        int32
dtype: object

In [27]:
categorical_data.dtypes

respondent_id         int32
region               object
gender               object
highest_education    object
employment_status    object
occupation           object
genre                object
dtype: object

In [28]:
numeric_data.dtypes

respondent_id                                                                  int32
Age                                                                            int32
household_size                                                                 int32
salary                                                                         int32
canadian_podcast_ratio_listen                                                  int32
us_podcast_ratio_listen                                                        int32
britain_podcast_ratio_listen                                                   int32
french_podcast_ratio_listen                                                    int32
elsewhere_podcast_ratio_listen                                                 int32
qe1ba_.%by_myself                                                              int32
qe1bb_.%with_others                                                            int32
qe2a_.%at_home                                                   

In [30]:
#Export all our cleaned data
kyc_data.to_csv('kyc_data.csv', index= False)
categorical_data.to_csv('cat_data.csv', index= False)
numeric_data.to_csv('num_data.csv', index= False)